In [69]:
from sklearn.datasets import fetch_20newsgroups
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
from nltk.stem import PorterStemmer
import numpy as np
import nltk
import pandas as pd
import os
nltk.download('names')
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()
all_names = names.words()
WNL = WordNetLemmatizer()

path_train= "/home/antonis/Downloads/20news-bydate/20news-bydate-train"
path_test = "/home/antonis/Downloads/20news-bydate/20news-bydate-test"


def gather(path):
    df = pd.DataFrame()
    for file in os.listdir(path):
        tag = file
        for doc in os.listdir(path+'/'+file):
            docpath = path+'/'+file+'/'+doc
            f = open(docpath, "r",encoding='cp1252')
            content = f.read()
            temp = pd.DataFrame(
                {
                    'content':content,
                    'tag':tag
                },index=[0]
            )
            df = pd.concat([df, temp])



    df.content =df.content.replace(to_replace='From:(.*\n)', value='', regex=True) ##remove from to email
    df.content =df.content.replace(to_replace='lines:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='Subject:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]', value=' ', regex=True) #remove punctuation except
    df.content =df.content.replace(to_replace='-', value=' ', regex=True)
    df.content =df.content.replace(to_replace='\s+', value=' ', regex=True)    #remove new line
    df.content =df.content.replace(to_replace='  ', value='', regex=True)                #remove double white space
    df.content =df.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

    df['content']=[entry.lower() for entry in df['content']]
    return df

df_news_train = gather(path_train)
df_news_test = gather(path_test)


def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                words = ps.stem(words)
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1
    return(list(cleaned.values()))

x_train = clean(df_news_train['content'])
x_test = clean(df_news_test['content'])


tf = TfidfVectorizer(stop_words='english', max_features=8000)
X_train = tf.fit_transform(x_train)




[nltk_data] Downloading package names to /home/antonis/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [70]:
print(tf.vocabulary_)

{'nntp': 4773, 'post': 5384, 'host': 3255, 'organ': 4982, 'comput': 1403, 'scienc': 6193, 'depart': 1837, 'line': 4054, 'need': 4686, 'spec': 6596, 'variou': 7507, 'data': 1718, 'format': 2660, 'intel': 3517, 'hex': 3165, 'motorola': 4566, 'anyon': 308, 'provid': 5550, 'info': 3461, 'pointer': 5343, 'want': 7658, 'mike': 4425, 'univers': 7396, 'illinoi': 3355, 'urbana': 7441, 'john': 3707, 'write': 7845, 'hat': 3091, 'did': 1906, 'pull': 5570, 'thi': 7085, 'db': 1732, 'ratio': 5702, 'unit': 7393, 'current': 1658, 'amper': 250, 'french': 2714, 'man': 4214, 'studi': 6796, 'electr': 2216, 'term': 7041, 'amp': 249, 'just': 3763, 'abbrevi': 5, 'compani': 1369, 'came': 990, 'wa': 7634, 'alreadi': 215, 'don': 2046, 'know': 3867, 'doesn': 2036, 'sound': 6568, 'got': 2917, 'thing': 7088, 'right': 5976, 'hello': 3138, 'oh': 4907, 'jeff': 3671, 'jeffrey': 3673, 'curti': 1661, 'power': 5402, 'revel': 5929, 'role': 6032, 'self': 6258, 'appoint': 332, 'soni': 6557, 'phase': 5233, 'linear': 4055, 'cr

In [73]:
vocabulary = tf.vocabulary_
print(vocabulary)

{'nntp': 4773, 'post': 5384, 'host': 3255, 'organ': 4982, 'comput': 1403, 'scienc': 6193, 'depart': 1837, 'line': 4054, 'need': 4686, 'spec': 6596, 'variou': 7507, 'data': 1718, 'format': 2660, 'intel': 3517, 'hex': 3165, 'motorola': 4566, 'anyon': 308, 'provid': 5550, 'info': 3461, 'pointer': 5343, 'want': 7658, 'mike': 4425, 'univers': 7396, 'illinoi': 3355, 'urbana': 7441, 'john': 3707, 'write': 7845, 'hat': 3091, 'did': 1906, 'pull': 5570, 'thi': 7085, 'db': 1732, 'ratio': 5702, 'unit': 7393, 'current': 1658, 'amper': 250, 'french': 2714, 'man': 4214, 'studi': 6796, 'electr': 2216, 'term': 7041, 'amp': 249, 'just': 3763, 'abbrevi': 5, 'compani': 1369, 'came': 990, 'wa': 7634, 'alreadi': 215, 'don': 2046, 'know': 3867, 'doesn': 2036, 'sound': 6568, 'got': 2917, 'thing': 7088, 'right': 5976, 'hello': 3138, 'oh': 4907, 'jeff': 3671, 'jeffrey': 3673, 'curti': 1661, 'power': 5402, 'revel': 5929, 'role': 6032, 'self': 6258, 'appoint': 332, 'soni': 6557, 'phase': 5233, 'linear': 4055, 'cr

In [96]:
def gen_vector(tokens)
    Q = np.zeros((len(vocabulary)))
    tf = TfidfVectorizer(stop_words='english', max_features=8000,vocabulary=vocabulary)
    print(Q)
    x= tf.fit_transform(tokens)
    #print(tokens[0].split(','))
    for token in tokens[0].split(','):
        #print(token)
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q


[0. 0. 0. ... 0. 0. 0.]



In [99]:
tokens = ['computer','science']
for token in tokens[0].split(','):
    ind = vocabulary.index(token)
    print(ind)

AttributeError: 'dict' object has no attribute 'index'

TypeError: 'Series' objects are mutable, thus they cannot be hashed